In [1]:
# 
# import json
# from datetime import datetime
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Download the webpage

In [2]:
url = "https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/Monkeypox-Data.aspx"

In [3]:
page = requests.get(url)

### Parse the webpage

In [4]:
soup = BeautifulSoup( page.content, 'html.parser' )

In [5]:
content = soup.find("div", id="DeltaPlaceHolderMain")

In [6]:
tables = content.find_all("table")

In [7]:
len(tables)

8

In [8]:
tables[2].find( "th" ).find("strong").text

'By Local\r\n  Health Jurisdiction'

In [37]:
for tag in content.find_all(text=re.compile('Jurisdiction')):
    table = tag.findParent("table")

In [38]:
def safetxt(element):
    v = element.text.strip()
    v = v.replace("\u200b", "")
    return v

In [39]:
def safenumber(element):
    v = safetxt(element)
    v = v.replace(",", "")
    v = v.replace(" ", "")
    return v

In [52]:
dict_list = []

for row in table.find_all("tr"):
    cell_list = row.find_all("td")
    #print(cell_list)
    try:
        d = dict(
            lhj_name = safetxt( cell_list[0] ),
            cases = safenumber( cell_list[1] )        
        )
        dict_list.append(d)
    except Exception as e: 
        print(e)

list index out of range


### Parsing date

In [79]:
soup = BeautifulSoup( page.content, 'html.parser' )

In [86]:
content_date = soup.find_all("head")

In [113]:
#print(content_date)

In [ ]:
"""-- Global site tag (gtag.js) - Google Analytics 
        <script async src="https://www.googletagmanager.com/gtag/js?id=UA-109056085-1"></script>
        <script>
            window.dataLayer = window.dataLayer || [];
            function gtag() { dataLayer.push(arguments); }
            gtag('js', new Date());
            
            gtag('config', 'UA-109056085-1');
        </script>-->"""

In [111]:
date_one = soup.find('script async src="https://www.googletagmanager.com/gtag/js?id=UA-109056085-1"')

In [112]:
print(date_one)

None


### Convert to dataframe

In [46]:
df = pd.DataFrame(dict_list)

### Export

In [49]:
df.to_csv("raw/cdph/latest.csv", index=False)